<a href="https://colab.research.google.com/github/parmarsuraj99/10DaysofMLChallenge/blob/master/day3/Day3_10daysofMLChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
!wget https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv

--2020-03-25 10:00:04--  https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ggl9krhh6dcwhhz/train.csv [following]
--2020-03-25 10:00:04--  https://www.dropbox.com/s/raw/ggl9krhh6dcwhhz/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ffa7d6f65d75ef4f0c4874345.dl.dropboxusercontent.com/cd/0/inline/A0kseGQUw-8KOQnvvUbZtEU2rE5Wxr6612SbHzBaptYb8sm8oycwRexYzmWkXkFgKMCHXI_Rzga0HBWAogOG7oQEamHzYbB4wV4ol8Qmc3p90w/file# [following]
--2020-03-25 10:00:04--  https://uc1ffa7d6f65d75ef4f0c4874345.dl.dropboxusercontent.com/cd/0/inline/A0kseGQUw-8KOQnvvUbZtEU2rE5Wxr6612SbHzBaptYb8sm8oycwRexYzmWkXkFgKMCHXI_Rzga0HBWAogOG7oQEamHzYbB4wV4ol8Qmc3p90w/file
Resolving uc1ffa7d6f65d75ef4

In [14]:
!pip install transformers
!pip install poutyne

     |████████████████████████████████| 81kB 3.6MB/s 


In [0]:
import os
import glob
import joblib
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm


In [0]:
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

from transformers import DistilBertTokenizer, DistilBertModel


In [0]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
train.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [0]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertModel.from_pretrained('distilbert-base-cased')

In [0]:
class TextClassifier(nn.Module):
    def __init__(self, n_labels, tokenizer, base_model):
        super().__init__()
        self.n_labels = n_labels
        self.tokenizer = tokenizer
        self.base_model = base_model
        self.classifier = nn.Linear(768, self.n_labels)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        
        outputs = self.base_model(input_ids)
        last_hidden_states = outputs[0][:, 0]
        x = self.dropout(last_hidden_states)
        x = self.classifier(x)
        x = torch.sigmoid(x)
        return x


In [0]:
net = TextClassifier(6, tokenizer, model)

In [0]:
%%time
#net("Hello")

CPU times: user 10.9 ms, sys: 1.98 ms, total: 12.9 ms
Wall time: 12.3 ms


tensor([[-0.7461,  0.7159, -0.6606, -0.1251, -0.3487]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [14]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [15]:
%%time
train_tokens = tokenizer.batch_encode_plus(train["comment_text"].values, 
                                           add_special_tokens=True, max_length=512,
                                           pad_to_max_length = True)

CPU times: user 3min 19s, sys: 692 ms, total: 3min 19s
Wall time: 3min 19s


In [16]:
%%time
test_tokens = tokenizer.batch_encode_plus(list(test["comment_text"].values), 
                                          add_special_tokens=True, max_length=512,
                                          pad_to_max_length = True)

CPU times: user 1min 20s, sys: 267 ms, total: 1min 20s
Wall time: 1min 20s


In [17]:
joblib.dump(train_tokens, "train_tokens.pkl")

['train_tokens.pkl']

In [18]:
joblib.dump(test_tokens, "test_tokens.pkl")

['test_tokens.pkl']

In [0]:
#train_tokens = joblib.load("/content/train_tokens.pkl")
#test_tokens = joblib.load("/content/test_tokens.pkl")

In [0]:
train["tokens"] = train_tokens["input_ids"]
test["tokens"] = test_tokens["input_ids"]

In [186]:
train_ids = []
for i, token in tqdm(enumerate(train_tokens["input_ids"])):
    train_ids.append(np.asarray(token))

test_ids = []
for i, token in tqdm(enumerate(test_tokens["input_ids"])):
    test_ids.append(np.asarray(token))

In [0]:
dep_vars = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
features = ["comment_text"]

In [58]:
f_model.to("cpu")

In [0]:
x_train = torch.tensor(train_ids)
y_train = torch.tensor(train[dep_vars].astype(float).values)

x_test = torch.tensor(test_ids)
y_test = torch.tensor(test[dep_vars].astype(float).values)

assert x_train.shape[0] == y_train.shape[0]
assert x_test.shape[0] == y_test.shape[0]

In [0]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
dataset = TensorDataset(x_train, y_train)
test_set = TensorDataset(x_test, y_test)

In [0]:
loader = data.DataLoader(
    dataset,
    batch_size=64
)
test_loader = data.DataLoader(
    test_set,
    batch_size=64
)

In [176]:
import gc
torch.cuda.empty_cache()
gc.collect()

355

In [168]:
x_train.shape[0]//64

4986

In [0]:
net.to("cuda");

In [0]:
net.train()
#Freezing DistillBERT weights! 
for param in net.base_model.parameters():
    param.requires_grad = False

In [0]:
for epoch in range(2):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data_t in tqdm(enumerate(loader), total=x_train.shape[0]//64):
        #print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data_t

        optimizer.zero_grad()

        inputs = inputs.to("cuda")
        labels = labels.to(dtype=torch.float)
        labels = labels.to("cuda")

        # zero the parameter gradients
        

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

In [193]:
with torch.no_grad():
    preds = []
    eval_loss = 0.0
    for i, data_t in tqdm(enumerate(test_loader), total=x_test.shape[0]//64):
        #print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data_t

        inputs = inputs.to("cuda")
        labels = labels.to(dtype=torch.float)
        labels = labels.to("cuda")

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        preds.append(outputs)

        # print statistics
        eval_loss += loss.item()

    print("Eval loss:", eval_loss/(x_test.shape[0]//64))



Eval loss: 0.0


In [196]:
eval_loss/(x_test.shape[0]//64)

0.09385954058944462

In [0]:
preds = torch.cat(preds)

In [198]:
preds.shape

torch.Size([63978, 6])

In [199]:
dep_vars

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
preds = preds.cpu()

In [0]:
preds_b = preds

In [0]:
preds_b[preds_b>0.5] = 1

In [0]:
preds_b[preds_b<0.5] = 0

In [224]:
for i in range(len(dep_vars)):
    print(f"Accuracy for {dep_vars[i]} : {accuracy_score(y_test[:, i], preds_b[:, i])}")

ERROR! Session/line number was not unique in database. History logging moved to new session 71
Accuracy for toxic : 0.9010441089124387
Accuracy for severe_toxic : 0.994248022757823
Accuracy for obscene : 0.9496076776391885
Accuracy for threat : 0.9967019913095126
Accuracy for insult : 0.9477320328863047
Accuracy for identity_hate : 0.9888711744662227
